In [37]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
import os
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.pdfbase import pdfmetrics

# 크롬 드라이버 자동 업데이트
chrome_driver_path = ChromeDriverManager().install()

# Selenium 웹 드라이버 설정
# chrome_options = Options()
# chrome_options.add_argument("--headless")
# chrome_options.add_argument("--disable-gpu")
# service = Service(executable_path=chrome_driver_path)
# driver = webdriver.Chrome(service=service, options=chrome_options)
driver = webdriver.Chrome()

# PDF 파일 생성
pdf_file_path = "researches/pdfs/real_estate.pdf"
c = canvas.Canvas(pdf_file_path, pagesize=letter)

# 한글 폰트 설정
font_path = "./font/NanumGothic.ttf"
pdfmetrics.registerFont(TTFont("Korean", font_path))

# 1. https://omoney.kbstar.com/quics?page=C042015&cc=b037806:b037806에 진입한다.
url = "https://omoney.kbstar.com/quics?page=C042015&cc=b037806:b037806"
driver.get(url)
time.sleep(1)

# 1페이지부터 5페이지까지 반복
for page_num in range(1, 6):

    # 2. html에서 class="tbl_list"안에서 href 목록과 제목을 구하는 코드
    tbl_list = driver.find_element(By.CLASS_NAME, "tbl_list")
    links = tbl_list.find_elements(By.TAG_NAME, "a")
    titles = [link.text for link in links]
    hrefs = [link.get_attribute("href") for link in links]

    # 3. 각 href와 제목를 PDF 페이지로 추가
    for title, href in zip(titles, hrefs):
        # 타이틀을 먼저 그리기
        c.setFont("Korean", 10)  # 폰트 크기를 10으로 설정
        c.drawString(100, 700, f"Title: {title}")  # 타이틀 추가
        
        y_position = 670  # 내용 시작 위치

        # 게시글 방문
        driver.get(href)
        time.sleep(1)  # 페이지 로딩 대기
        
        # 등록일 정보 크롤링
        date_elements = driver.find_elements(By.CLASS_NAME, "date")
        date_text = ""
        for date_element in date_elements:
            em_element = date_element.find_element(By.TAG_NAME, "em")
            if em_element.text == "등록일":
                date_text = date_element.text.replace("등록일", "").strip()
                break
                
        c.drawString(100, y_position, f"등록일: {date_text}")  # 등록일 추가
        y_position -= 12  # 줄 간격 조절
        
        view_cont = driver.find_element(By.ID, "view_cont").text

        # 줄바꿈을 처리하고, 빈 줄인 경우는 제외
        lines = view_cont.split("\n")
        lines = [line for line in lines if line.strip() != '']

        # 페이지별로 PDF 페이지에 추가
        for line in lines:
            if y_position < 50:  # 페이지 내용이 공간이 다 차면 다음 페이지로
                c.showPage()  # 페이지 추가
                c.setFont("Korean", 10)  # 폰트 크기 재설정
                c.drawString(100, 700, f"Title: {title}")  # 타이틀 추가
                y_position = 670  # 내용 시작 위치 재설정
                c.drawString(100, y_position, f"등록일: {date_text}")  # 등록일 추가
                y_position -= 12  # 줄 간격 조절
            c.drawString(100, y_position, line)  # 내용 추가
            y_position -= 12  # 줄 간격 조절

        y_position -= 12  # 줄 간격 조절
        c.showPage()  # 페이지 추가

    # 초기 페이지로 가기
    driver.get(url)
    time.sleep(1)
    # 페이지 넘어가는 버튼 클릭
    if page_num < 5:
        next_page_num = page_num + 1
        next_page_button = driver.find_element(By.XPATH, f'//*[@id="pageinput{next_page_num}"]/span/input')
        next_page_button.click()
        time.sleep(1)  # 페이지 로딩 대기

driver.quit()

# 기존의 real_estate.pdf 파일 삭제
if os.path.exists(pdf_file_path):
    os.remove(pdf_file_path)

c.save()  # PDF 파일 저장
